In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random

Intitializing Scala interpreter ...

Spark Web UI available at http://b40790111e29:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1618501230566)
SparkSession available as 'spark'


2021-04-15 15:40:29 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark.SparkConf
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random


In [2]:
class VertexAttr(
    var degree : Int = 0,
    var initialLabel : Boolean = false,
    var vertexLabels : Array[Float] = Array[Float](0, 0, 0)
) extends java.io.Serializable {     
    
    def setDegree(degree: Int) = {
        this.degree = degree
    }
}

defined class VertexAttr


In [3]:
val rand = new Random(42)

def random_label(random: Random): Int = {
    random.nextInt(3)
}

def transform(VD: Int): VertexAttr = {
    val label = random_label(rand);
    var va = new VertexAttr();
    
    if (rand.nextInt(2) == 1) {
        if (label == 0) {
            va.initialLabel = true;
            va.vertexLabels = Array[Float](1, 0, 0);
        } else if (label == 1) {
            va.initialLabel = true;
            va.vertexLabels = Array[Float](0, 1, 0);
        } else if (label == 2) {
            va.initialLabel = true;
            va.vertexLabels = Array[Float](0, 0, 1);
        }
    }
    va;
}

rand: scala.util.Random = scala.util.Random@18fb517b
random_label: (random: scala.util.Random)Int
transform: (VD: Int)VertexAttr


In [4]:
// Load initial Graph
val initialGraph = GraphLoader.edgeListFile(sc, "./soc-karate/soc-karate.mtx", canonicalOrientation=false)

initialGraph: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@6e5b003b


In [5]:
// Transform vertices into VertexAttr with a random label assigned
val graph = initialGraph.mapVertices({ case (id, attr) => transform((attr))})

graph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@5b460bfb


In [6]:
// Populate Graph degree for each vertices and create final graph
val vertex_rdd = graph.degrees.zip(graph.vertices).map({ case (vDeg, vAttr) => vAttr._2.setDegree(vDeg._2) ; vAttr})
val edge_rdd = graph.edges

val graph2 = Graph(vertex_rdd, edge_rdd)

// Edges go both sides ways
val finalGraph = Graph(graph2.vertices, graph2.edges.union(graph2.edges.reverse))

vertex_rdd: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, VertexAttr)] = MapPartitionsRDD[20] at map at <console>:51
edge_rdd: org.apache.spark.graphx.EdgeRDD[Int] = EdgeRDDImpl[14] at RDD at EdgeRDD.scala:41
graph2: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@3e7e2058
finalGraph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@679b4bc2


In [7]:
// Vertex Progam
def VertexProgram(id : VertexId, 
                  VD : VertexAttr, 
                  A : Array[Float]) : VertexAttr  = {
    var newVertexAttr = new VertexAttr()
    newVertexAttr.degree = VD.degree
    newVertexAttr.initialLabel = VD.initialLabel
    newVertexAttr.vertexLabels = VD.vertexLabels
    
    if(VD.initialLabel == false){
        newVertexAttr.vertexLabels = A.map(_ / VD.degree)
    }
    newVertexAttr
}

// Send Message
def SendMsg(triplet : EdgeTriplet[VertexAttr, Int]): Iterator[(VertexId, Array[Float])] = {
//     println(triplet.srcId, "->", triplet.dstId, triplet.srcAttr.vertexLabels.mkString(", "))
    Iterator((triplet.dstId, triplet.srcAttr.vertexLabels))
}

// Merge Message
def MergeMsg(A : Array[Float],
             B : Array[Float]) : Array[Float] = {
    val mergeMsg = new Array[Float](A.length);
    for ( i <- 0 to (A.length - 1)) {
         mergeMsg(i) = A(i) + B(i);
    }
    mergeMsg;
}

VertexProgram: (id: org.apache.spark.graphx.VertexId, VD: VertexAttr, A: Array[Float])VertexAttr
SendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[VertexAttr,Int])Iterator[(org.apache.spark.graphx.VertexId, Array[Float])]
MergeMsg: (A: Array[Float], B: Array[Float])Array[Float]


In [8]:
val initialMsg = Array[Float](0, 0, 0)
val maxIter = 100

val graphLP = finalGraph.pregel(initialMsg, maxIter, EdgeDirection.Either)(
    (id, VD, newLabelsArray) => VertexProgram(id, VD, newLabelsArray),
    tr => SendMsg(tr),
    (a, b) => MergeMsg(a, b)
)

initialMsg: Array[Float] = Array(0.0, 0.0, 0.0)
maxIter: Int = 100
graphLP: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@20bb4c03


In [9]:
graphLP.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

(13,2,false,0.058400773 0.30003047 0.64156866)
(19,2,false,0.7078702 0.08155619 0.21057361)
(15,2,true,1.0 0.0 0.0)
(21,2,true,0.0 0.0 1.0)
(25,3,false,0.2843869 0.016873697 0.69873935)
(29,3,false,0.13858013 0.054370794 0.8070491)
(11,3,false,0.027413137 0.6894467 0.28314006)
(27,2,true,0.0 0.0 1.0)
(33,12,true,1.0 0.0 0.0)
(23,2,true,0.0 1.0 0.0)
(1,16,false,0.054826275 0.37889344 0.5662801)
(17,2,false,0.0 1.0 0.0)
(3,10,true,0.0 0.0 1.0)
(7,4,true,0.0 1.0 0.0)
(9,5,false,0.38856718 0.13522969 0.47620305)
(31,4,false,0.47226927 0.13414264 0.39358807)
(5,3,false,0.027413137 0.6894467 0.28314006)
(34,17,false,0.41574037 0.16311239 0.42114723)
(4,6,false,0.061975274 0.2211675 0.71685714)
(16,2,true,1.0 0.0 0.0)
(22,2,true,0.0 0.0 1.0)
(28,4,false,0.4250318 0.044996522 0.52997166)
(30,4,false,0.6039351 0.040778097 0.3552868)
(14,5,false,0.1234623 0.20028035 0.6762573)
(32,6,true,0.0 0.0 1.0)
(24,5,true,1.0 0.0 0.0)
(6,4,true,0.0 1.0 0.0)
(8,4,false,0.05039277 0.20957235 0.7400348)
(12,1